In this project i will work with data from the **CIA World Factbook**, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following as  per the year 2015:

* population — the global population.
* population_growth — the annual population growth rate, as a percentage.
* area — the total land and water area.

Below cell has the code which connect our Jupyter Notebook to our database:

In [10]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Finding out the information about different tables in the database using below SQL

In [11]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Finding out the **first 5 rows of the Facts table** in the database

In [12]:
%%sql
SELECT *
  FROM facts LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The column names are mostly self-explanatory; below are the descriptions for those less evident:

* area_land – the country's land area in km2,
* area_water – the country's water area in km2,
* population_growth– the country's population growth in %,
* birth_rate – the number of births per year per 1,000 people,
* death_rate – the number of deaths per year per 1,000 people,
* migration_rate – the difference between the number of persons entering (immigrants) and leaving (emigrants) a country during the year, per 1,000 people.

Let's calculate some summary statistics for population and look for any outlier countries. Querying to find the **minimum and maximum population and population growth**:

* Minimum population
* Maximum population
* Minimum population growth
* Maximum population growth

In [20]:
%%sql
SELECT
MIN(population) AS 'Least Populated', 
MAX(population) AS 'Most Populated', 
MIN(population_growth) AS 'Least Population growth', 
MAX(population_growth) AS 'Most Population growth' 
FROM facts;

Done.


Least Populated,Most Populated,Least Population growth,Most Population growth
0,7256490011,0.0,4.02


There are few interesting things in the summary statistics above:

1. There's a country with a population of 0
2. There's a country with a population of "7256490011" (or more than 7.2 billion people) which is almost the world population.

Let's find the names of these countries dynamically

In [40]:
%%sql
SELECT
Name, population
FROM facts
WHERE population = (SELECT MIN(population) FROM facts) 
OR 
population = (SELECT MAX(population) FROM facts);

Done.


name,population
Antarctica,0
World,7256490011


We can confirm that the maximum value of population is just related to the world population in 2015, represented by its own row in the table. 

The minimum value, instead, is related to Antarctica, which also looks reasonable.

Let's recalculate the summary statistics **excluding Antartica and World**.

In [37]:
%%sql
SELECT
MIN(population) AS 'Least Populated',
MAX(population) AS 'Most Populated',
ROUND(AVG(population)) AS 'Avg Population'
FROM facts
WHERE name != 'Antarctica' 
AND 
name != 'World';

Done.


Least Populated,Most Populated,Avg Population
48,1367485388,32377011.0


Now the minimum and maximum values of the population look much more menaningful.

Also, the average population without the world and Antarctica has been decreased almost twice.

In [41]:
%%sql
SELECT
AVG(population) AS 'AVG Population',
AVG(area) AS 'AVG Area'
FROM facts
WHERE name != 'Antarctica' 
AND 
name != 'World';


Done.


AVG Population,AVG Area
32377011.0125,555093.546184739


Now lets find out what are the countries whoes "population is above the average population **AND** area is below the average area".

In [47]:
%%sql
SELECT Name 
FROM facts 
WHERE population > 32377011.0125
AND
area < 555093.546184739;

Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


Second method of finding the countries which matches above criteria using Sub queries.

In [49]:
%%sql
SELECT Name 
FROM facts 
WHERE population > (SELECT AVG(population) FROM facts 
                    WHERE name != 'Antarctica' AND name != 'World')
AND
area < (SELECT AVG(area) FROM facts 
                    WHERE name != 'Antarctica' AND name != 'World')

Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


Interesting! 

Looks like Bangladesh has the population which is greater than average population average with lower area than average.


### The most populated country..###

Lets find the most populated country

In [56]:
%%sql
SELECT Name,
population FROM facts
WHERE population = (SELECT MAX(population) FROM facts
                   WHERE name != 'Antarctica' AND name != 'World');

Done.


name,population
China,1367485388


### The highest growth rate country..###

Lets find the country which has highest growth rate

In [58]:
%%sql
SELECT Name, 
population_growth FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts
                          WHERE name != 'World' AND name != 'Antarctica');

Done.


name,population_growth
South Sudan,4.02
